In [ ]:
from datetime import datetime
import numpy as np
import os
from Bio import SeqIO
import importlib
import sys
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import func_initiation as f

Create a species_map to acquire the location of species in data frame/ CSV file

In [ ]:
species, ini_counts = f.speciesandIniCounts()

species_map = {}

for i_specie, specie in enumerate(species):
    species_map[specie] = i_specie

print(species_map)


Read in the CSV files from multiple replicates into two 3D arrays for with or without volume change (species, time, replicates)

In [ ]:
# The written time of whole cell is per second in 105 minutes.
endtime = 105
simtime = 60
reps = 50
len_species = 74

WCCtimespan = np.linspace(0,endtime*simtime,endtime*simtime+1)

data_with = np.zeros((len_species,len(WCCtimespan), reps))

# Read csv file into one numpy array
for rep in range(reps):
    
    trajDF = pd.read_csv('./WithVolumeChange/initiation_{0}.csv'.format(rep+1),index_col=0)
    
    # index_col = 0 to skip the index
    data_with[:,:,rep] = trajDF.to_numpy()


data_without = np.zeros((len_species,len(WCCtimespan), reps))

for rep in range(reps):
    
    trajDF = pd.read_csv('./WithoutVolumeChange/initiation_{0}.csv'.format(rep+1),index_col=0)
    
    # index_col = 0 to skip the index
    data_without[:,:,rep] = trajDF.to_numpy()

print('The dimensions of trajectories with volume change is {0}'.format(np.shape(data_with)))
print('The dimensions of trajectories without volume change is {0}'.format(np.shape(data_without)))

### With Volume Change

Plot the traces of marker species RepInitCheck; Use RepInitCheck to determine whether initiation finished or not

Plot the filament length of several replicates

In [ ]:
# With Volume Change
# spcIdx = sim.particleMap['RepInitCheck'] -1 
spcIdx = species_map['RepInitCheck']

uninitialized_list = []

for rep in range(reps):

    InitCheck = data_with[spcIdx,:,rep]

    if InitCheck[-1] < 1:
        uninitialized_list.append(rep+1)
    
    plt.plot(WCCtimespan/60, InitCheck)

MeanInitCheck = np.mean(data_with[spcIdx,:,:], axis = 1)

plt.plot(WCCtimespan/60, MeanInitCheck, label = 'Mean')

plt.legend()

print(len(uninitialized_list), uninitialized_list)


In [ ]:
uninitialized_length = []

for rep in uninitialized_list:
    fila = np.zeros(len(WCCtimespan))
    for i in range(0,30):
        bnd = 'ssDNAunboundSite_' + str(i+1)
        spcIdx = species_map[bnd]
        # The DNA filament length calculated in a smart way
        fila += data_with[spcIdx,:,rep-1 ]*(i+1)
     
    uninitialized_length.append(np.max(fila))

print(uninitialized_list, uninitialized_length)
print('{0} out of {1} replicates failed to initialize DNA replication and their dnaA filament average length is {2}'.format(len(uninitialized_list), reps, np.mean(uninitialized_length)))

### Without Volume Change

Plot the traces of marker species RepInitCheck; Use RepInitCheck to determine whether initiation finished or not

Plot the filament length of several replicates

In [ ]:
# Without Volume Change
# spcIdx = sim.particleMap['RepInitCheck'] -1 
spcIdx = species_map['RepInitCheck']

uninitialized_list = []

for rep in range(reps):

    InitCheck = data_without[spcIdx,:,rep]

    if InitCheck[-1] < 1:
        uninitialized_list.append(rep+1)
    
    plt.plot(WCCtimespan/60, InitCheck)

MeanInitCheck = np.mean(data_without[spcIdx,:,:], axis = 1)

plt.plot(WCCtimespan/60, MeanInitCheck, label = 'Mean')

plt.legend()

print(len(uninitialized_list), uninitialized_list)

In [ ]:
uninitialized_length = []
for rep in uninitialized_list:
    fila = np.zeros(len(WCCtimespan))
    for i in range(0,30):
        bnd = 'ssDNAunboundSite_' + str(i+1)
        spcIdx = species_map[bnd] 
        # The DNA filament length calculated in a smart way
        fila += data_without[spcIdx,:,rep-1 ]*(i+1)
    uninitialized_length.append(np.max(fila))

print(uninitialized_list, uninitialized_length)
print('{0} out of {1} replicates failed to initialize DNA replication and their dnaA filament average length is {2}'.format(len(uninitialized_list), reps, np.mean(uninitialized_length)))